In [1]:

from dataclasses import dataclass
from typing import Dict, List, Tuple
from collections import defaultdict
import random
import logging

import numpy as np
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli, Statevector
from qiskit.circuit.library import UnitaryGate


from numba import njit, complex128, int64

from pauli_propagation import PauliTerm, PauliPropagator
from pauli_propagation.utils import random_su4, encode_pauli

In [2]:
np.random.seed(42)
def _staircase_edges(nx, ny):
    """Return the ordered list of 1-based index pairs of the staircase walk."""
    next_inds, temp_inds, edges = [1], [], []
    while next_inds:
        for ind in next_inds:
            if ind % nx != 0:                      # step right
                nxt = ind + 1
                edges.append((ind, nxt)); temp_inds.append(nxt)
            if ((ind-1)//nx + 1) < ny:             # step down
                nxt = ind + nx
                edges.append((ind, nxt)); temp_inds.append(nxt)
        next_inds, temp_inds = temp_inds, []
    seen, uniq = set(), []
    for e in edges:                                # preserve order, dedup
        if e not in seen:
            seen.add(e); uniq.append(e)
    return uniq

def staircasetopology2d_qc(nx, ny, L):
    """
    Build a QuantumCircuit that places a fresh SU(4) gate on every edge
    of the 2-D staircase topology for an nx × ny grid (row-major indexing).
    """
    nqubits = nx * ny
    qc = QuantumCircuit(nqubits)
    for _ in range(L):
        for k, (q1, q2) in enumerate(_staircase_edges(nx, ny)):  # 1-based
            mat = random_su4()
            gate = UnitaryGate(mat, label=f"SU4_{k}")   # create the gate
            gate._name = "su4"                          # override its name
            qc.append(gate, [q1-1, q2-1])               # 0-based
    return qc

# Example usage:
L = 1
nx, ny = 5,5
qc_2d = staircasetopology2d_qc(nx, ny, L)

# fig = qc_2d.draw(output="mpl", fold=-1)   # no line-wrapping
# plt.show()

In [3]:
n = qc_2d.num_qubits # Here n is number of qubit in 2d circuit

plus_label = "+" * n # build a string of "+" of length n, e.g. "+++++"

# ─── Build the single-qubit X observable on qubit 0 ───────────────────────────
pauli_label = "X" + "I" * (n - 1) # For X on qubit 0 and Identity elsewhere, we need "X" + "I"*(n-1)
obs = Pauli(pauli_label)

In [4]:
# 5*5, 1 layer, k = 3, 12s
# 5*5, 1 layer, k =4, 153s
# 5*5, 2 layers, k = 3, 47s

# 6,6, 1 layer, k = 3, 29.5s
# 6,6, 2 layers, k = 3, 123s

In [5]:
# 感觉精度变差了，尝试提高精度

prop_2d   = PauliPropagator(qc_2d)
print('1')

# 把字符串 → qiskit.Pauli → 位掩码
key_obs = encode_pauli(Pauli(pauli_label))

# 新构造方式：PauliTerm(coeff, key, n)
obs_term = PauliTerm(1.0, key_obs, qc_2d.num_qubits)

# 反向传播（可选 max_weight）
layers_2d = prop_2d.propagate(obs_term, max_weight = 3)
print('2')
# 计算期望值
plus_label = "+" * qc_2d.num_qubits         # |+⟩^{⊗n}
expect_2d  = prop_2d.expectation_pauli_sum(layers_2d[-1], plus_label)
print('3')
print(expect_2d)

1
2
3
0.12079133418758366


In [6]:
# ─── Evolve the statevector through the 2D staircase circuit ─────────────────
psi0 = Statevector.from_label(plus_label)
psi_final = psi0.evolve(qc_2d)

# ─── Compute and print the expectation value ───────────────────────────────────
exp_val = psi_final.expectation_value(obs).real
print(f"⟨X on qubit 0⟩ from statevector simulation: {exp_val:.6f}")

⟨X on qubit 0⟩ from statevector simulation: 0.157547


In [8]:
# 今日任务： 确认以前的方法和现在的方法给出一样的答案(已完成)
# 把pauli expectation方法也给numba/掩码化 (已完成)
# 整理成package (已完成)
# 跑通所有test suite (已完成)
# 
